# Pandas -- Series and DataFrames


Main source: https://github.com/jakevdp/PythonDataScienceHandbook/tree/master/notebooks

#### Pandas is a library for fast and efficient computation on big datasets. As in Numpy, many operations in Pandas are vectorized and thus efficient and fast.
Pandas is a newer package built on top of NumPy. It provides an efficient implementation of a DataFrame. DataFrames are essentially multidimensional numpy arrays with additionally attached row and column labels. They allow for heterogeneous types(unlike numpy arrays) and/or missing data and they offer a convenient storage interface for labeled data. Furthermore Pandas implements a number of powerful data operations familiar to users of both database frameworks (like relational algebra) and spreadsheet programs.

As we saw, NumPy's ndarray data structure provides essential features for the type of clean, well-organized data typically seen in numerical computing tasks. While it serves this purpose very well, its limitations become clear when we need more flexibility (e.g., attaching labels to data, working with missing data, etc.) and when attempting operations that do not map well to element-wise broadcasting (e.g., groupings, pivots, etc.), each of which is an important piece of analyzing the less structured data available in many forms in the world around us. Pandas, and in particular its Series and DataFrame objects, builds on the NumPy array structure and provides efficient access to these sorts of "data munging" tasks that occupy much of a data scientist's time.

In [1]:
# Just as we import numpy usually as np, we import pandas under the alias of pd. 
# We'll import numpy as well, because we'll need it often when using pandas
import numpy as np
import pandas as pd

## The Pandas Series Object
A Pandas Series is a one-dimensional array of indexed data. It can be created from a list or array as follows:

In [2]:
data = pd.Series([0.25, 5, np.NaN, 1.0])
data

0    0.25
1    5.00
2     NaN
3    1.00
dtype: float64

In [3]:
type(data)

pandas.core.series.Series

In [4]:
data.values, type(data.values)

(array([0.25, 5.  ,  nan, 1.  ]), numpy.ndarray)

In addition to the values Pandas Series have an index which is an array-like object.

In [5]:
data.index, type(data.index), list(data.index)

(RangeIndex(start=0, stop=4, step=1),
 pandas.core.indexes.range.RangeIndex,
 [0, 1, 2, 3])

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [6]:
data[1:3]

1    5.0
2    NaN
dtype: float64

In [7]:
type(data[1])

numpy.float64

### Series as generalized NumPy array

From what we've seen so far, it may look like the Series object is basically interchangeable with a one-dimensional NumPy array. The essential difference is the presence of the index: while the Numpy Array has an implicitly defined integer index used to access the values, the Pandas Series has an explicitly defined index associated with the values.

This explicit index definition gives the Series object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type. For example, if we wish, we can use strings as an index:

In [8]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'd', 'c'])
data

a    0.25
b    0.50
d    0.75
c    1.00
dtype: float64

In [9]:
data.index = list("AbCD")
data

A    0.25
b    0.50
C    0.75
D    1.00
dtype: float64

In [10]:
data["b"] == data[1] == data.values[1]

True

In [11]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[3, 7, 2, 4])
data

3    0.25
7    0.50
2    0.75
4    1.00
dtype: float64

In [12]:
data.index

Index([3, 7, 2, 4], dtype='int64')

When an explicit index is present, it is preferred! (*as long as we don't slice!*)

In [13]:
data[3]

0.25

In [14]:
data[1:3]

7    0.50
2    0.75
dtype: float64

Note that explicit indices don't need to be unique!

In [15]:
d2 = pd.concat([data, data])
d2

3    0.25
7    0.50
2    0.75
4    1.00
3    0.25
7    0.50
2    0.75
4    1.00
dtype: float64

In [16]:
d2[3]

3    0.25
3    0.25
dtype: float64

In [17]:
d2[3] = 2
d2

3    2.00
7    0.50
2    0.75
4    1.00
3    2.00
7    0.50
2    0.75
4    1.00
dtype: float64

[Here you can find a exercise to pandas series](optional_exercises.ipynb#exe01a)
<img src="images/optional_exercises1.png" width="50" style="float: right;"/>

### Series as specialized dictionary

In this way, you can think of a Pandas Series a bit like a specialization of a Python dictionary. A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a Series is a structure which maps typed keys to a set of typed values. This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas Series makes it much more efficient than Python dictionaries for certain operations.

The Series-as-dictionary analogy can be made even more clear by constructing a Series object directly from a Python dictionary:

In [19]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [20]:
population['Texas']

26448193

Unlike a dictionary, though, the Series also supports array-style operations such as slicing:

In [21]:
population['California':'Illinois']
# note that Illinois is included!

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

### Constructing Series objects

In [22]:
# data can be a list or numpy array
pd.Series([2,4,5,6,7,8], index=list("abcdef"))

a    2
b    4
c    5
d    6
e    7
f    8
dtype: int64

In [23]:
# data can be a scalar
pd.Series(5, index=[100, 200, 300])

100    5
200    5
300    5
dtype: int64

In [24]:
# data can be a dictionary
ser = pd.Series({2:'a', 1:'b', 3:'c'})
ser

2    a
1    b
3    c
dtype: object

Notice that in this case, the Series is populated only with the explicitly identified keys:

In [25]:
pd.Series({2:'a', 1:'b', 3:'c'}, index=[3, 2])

3    c
2    a
dtype: object

[Here you can find a exercise to pandas series](optional_exercises.ipynb#exe01b)
<img src="images/optional_exercises1.png" width="50" style="float: right;"/>

<br>

## The Pandas DataFrame Object

The next fundamental structure in Pandas is the DataFrame. Like the Series object, it can be thought of either as a generalization of a NumPy array, or as a specialization of a Python dictionary. We'll now take a look at each of these perspectives.

### DataFrame as a generalized NumPy array

If a Series is an analog of a one-dimensional array with flexible indices, a DataFrame is an analog of a two-dimensional array with both flexible row indices and flexible column names. Just as you might think of a two-dimensional array as an ordered sequence of aligned one-dimensional columns, you can think of a DataFrame as a sequence of aligned Series objects. Here, by "aligned" we mean that they share the same index.



To demonstrate this, let's first construct a new Series listing the area of each of the five states discussed in the previous section:

In [26]:
area_dict = {'California': 423967, 
             'Texas': 695662, 
             'New York': 141297,
             'Florida': 170312, 
             'Illinois': 149995}

area = pd.Series(area_dict)
area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
dtype: int64

Now that we have this along with the population Series from before, we can use a dictionary to construct a single two-dimensional object containing this information:

In [27]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}

population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [28]:
states = pd.DataFrame({'population': population,
                       'area': area,
                       'country': 'USA'})
states

,population,area,country
California,38332521,423967,USA
Texas,26448193,695662,USA
New York,19651127,141297,USA
Florida,19552860,170312,USA
Illinois,12882135,149995,USA


In [29]:
print(states.dtypes)

population     int64
area           int64
country       object
dtype: object


This looks like a generalized dictionary! The keys are the names of the state, and the values are like a list [area, country, population]

In [30]:
states.sort_values(by="population", ascending=False)

,population,area,country
California,38332521,423967,USA
Texas,26448193,695662,USA
New York,19651127,141297,USA
Florida,19552860,170312,USA
Illinois,12882135,149995,USA


In [31]:
states['population'], type(states['population'])

(California    38332521
 Texas         26448193
 New York      19651127
 Florida       19552860
 Illinois      12882135
 Name: population, dtype: int64,
 pandas.core.series.Series)

In [32]:
states["population"].idxmax() #figures out the "key(s)"(indices) of the DataFrame where "population" has its max

'California'

In [33]:
states.loc[states["population"].idxmax()] #returnes the series at the given index

population    38332521
area            423967
country            USA
Name: California, dtype: object

In [34]:
states.max() #returns the max for every individual column!

population    38332521
area            695662
country            USA
dtype: object

In [35]:
states['California']

KeyError: 'California'

In [36]:
states.loc['California']

population    38332521
area            423967
country            USA
Name: California, dtype: object

[Here you can find a small exercise to test your understanding](optional_exercises.ipynb#exe02a)
<img src="images/optional_exercises1.png" width="50" style="float: right;"/>

In [37]:
states.index

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')

In [38]:
states.columns

Index(['population', 'area', 'country'], dtype='object')

In [39]:
states.values

array([[38332521, 423967, 'USA'],
       [26448193, 695662, 'USA'],
       [19651127, 141297, 'USA'],
       [19552860, 170312, 'USA'],
       [12882135, 149995, 'USA']], dtype=object)

In [40]:
type(states.values)

numpy.ndarray

Thus the DataFrame can be thought of as a generalization of a two-dimensional NumPy array, where both the rows and columns have a generalized index for accessing the data.

## DataFrame as specialized dictionary

Similarly, we can also think of a DataFrame as a specialization of a dictionary. Where a dictionary maps a key to a value, a DataFrame maps a column name to a Series of column data. For example, asking for the 'area' attribute returns the Series object containing the areas we saw earlier:

In [41]:
states["area"]
# note that indexing a DataFrame with square brackets gets the *column*!

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [42]:
type(states["area"])

pandas.core.series.Series

## Constructing DataFrame objects

A Pandas DataFrame can be constructed in a variety of ways:

#### From a single Series object

A DataFrame is a collection of Series objects, and a single-column DataFrame can be constructed from a single Series:

In [43]:
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [44]:
pd.DataFrame(population, columns=['population'])

,population
California,38332521
Texas,26448193
New York,19651127
Florida,19552860
Illinois,12882135


#### From multiple Series

In [45]:
s1 = pd.Series(list("abcd"))
s2 = pd.Series(list("efgh"))
df = pd.concat([s1, s2], axis="columns")
df

,0,1
0,a,e
1,b,f
2,c,g
3,d,h


Note that many functions in pandas take the `axis` argument - In which case you can select between 0/`index` and  1/`columns`. If you want to be explicit, I recommend using the string-version!

#### From a list of dicts 

Any list of dictionaries can be made into a DataFrame. We'll use a simple list comprehension to create some data. Even if some keys in the dictionary are missing, Pandas will fill them in with NaN (i.e., "not a number") values:

In [46]:
df = pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}], index=["first_dict", "second_dict"])
df

,a,b,c
first_dict,1.0,2,NaN
second_dict,NaN,3,4.0


As every single column must have a consistent dtype and np.NaN is a float, some of the numbers get coerced into floats:

In [47]:
df['a']

first_dict     1.0
second_dict    NaN
Name: a, dtype: float64

In [48]:
df['b']

first_dict     2
second_dict    3
Name: b, dtype: int64

In [49]:
df.dtypes

a    float64
b      int64
c    float64
dtype: object

If we wanted to get the rows, pandas would need to coerce the numbers explicitly: 

In [50]:
df

,a,b,c
first_dict,1.0,2,NaN
second_dict,NaN,3,4.0


In [51]:
df.loc['first_dict']

a    1.0
b    2.0
c    NaN
Name: first_dict, dtype: float64

#### From a two-dimensional NumPy array

Given a two-dimensional array of data, we can create a DataFrame with any specified column and index names. If omitted, an integer index will be used for each:

In [52]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2013-01-01,0.204666,-2.081420,-1.577676,0.689452
2013-01-02,0.264889,0.619927,-0.862270,1.051862
2013-01-03,1.042426,-0.604080,-1.296472,1.684356
2013-01-04,-1.599897,-0.180797,0.140971,-0.872624
2013-01-05,1.874404,-0.733691,-0.315427,0.069617
2013-01-06,-2.125060,0.578902,-0.049766,0.297027


[Here you can find a small exercise to test your understanding](optional_exercises.ipynb#exe02b)
<img src="images/optional_exercises1.png" width="50" style="float: right;"/>

## The Pandas Index Object

We have seen here that both the Series and DataFrame objects contain an explicit index that lets you reference and modify data. This Index object is an interesting structure in itself, and it can be thought of as an immutable array:

In [73]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Index([2, 3, 5, 7, 11], dtype='int64')

In [74]:
ind[0] = 1

TypeError: Index does not support mutable operations

In [75]:
sr = pd.Series(0, index=ind)
sr

2     0
3     0
5     0
7     0
11    0
dtype: int64

Index objects have a name:

In [76]:
ind.names = ['indexx']
ind

Index([2, 3, 5, 7, 11], dtype='int64', name='indexx')

In [77]:
sr = pd.Series(np.zeros_like(ind), index=ind)
sr

indexx
2     0
3     0
5     0
7     0
11    0
dtype: int64

In [78]:
df = pd.DataFrame(np.zeros_like(ind), index=ind, columns=['first'])
df

,first
indexx,
2,0
3,0
5,0
7,0
11,0


In [79]:
df.index.names = [None]
df

,first
2,0
3,0
5,0
7,0
11,0


Index objects also have many of the attributes familiar from NumPy arrays:

In [80]:
ind.size, ind.shape, ind.ndim, ind.dtype

(5, (5,), 1, dtype('int64'))

While viewing Indices as immutable list is natural, indices also allow for set-operations:

In [81]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [82]:
indA & indB

Index([0, 3, 5, 7, 9], dtype='int64')

In [83]:
indA | indB

Index([3, 3, 5, 7, 11], dtype='int64')

In [85]:
indA ^ indB

Index([3, 0, 0, 0, 2], dtype='int64')

[Here you can find a last exercise for this notebook](optional_exercises.ipynb#exe02c)
<img src="images/optional_exercises1.png" width="50" style="float: right;"/>